In [1]:
mail_flag = True

######################################################

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
from tqdm.notebook import tqdm
import telegram
import datetime as dt

import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.encoders import encode_base64
from email.header import Header
import getpass
import os



# headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3)'}

table = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
sp500_list = table[0]['Symbol'].tolist()
ticker_list = []
for x in sp500_list:
    ticker_list.append(x.replace('.', '-'))

options = webdriver.ChromeOptions()
options.headless = True
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3)")
    

df = pd.DataFrame(columns=['REVENUE', 'FS_DATE'])

dfTTL = pd.DataFrame()

for tk in tqdm(ticker_list, desc="S&P500 Rev Crawling", leave=True):
    print(tk)
    while True:
        driver = webdriver.Chrome(options=options)
        try:  
            
            url = 'https://finance.yahoo.com/quote/' + tk + '/financials?p=' + tk     
            driver.get(url)
            
            try:
                popup = WebDriverWait(driver, 5).until(EC.visibility_of_element_located\
                                                        ((By.XPATH, '//*[@id="myLightboxContainer"]/section/button[1]')))
                popup.click()
            except TimeoutException as to:
                print(to)
            
            
            
            
            
            driver.find_element_by_xpath("//span[text()='Quarterly']").click()
            time.sleep(3)
            soup = BeautifulSoup(driver.page_source, 'lxml')
            ls= []
            for l in soup.find_all('div'): 
                ls.append(l.string)
            ls_filtered = list(filter(None,ls))[14:]

            date= []
            sales = []
            op = []
            
            flag_date = False

            for no, x in enumerate(ls_filtered):
                x = x.replace(',','')
                if x == 'ttm':
                    no_start = no
                    flag_date = True
                if flag_date:
                    date.append(x)
                if (x.isdigit() or ('-' in x)) and flag_date==True:
                    flag_date=False
                    no_end = no
                    no_final = no_end-no_start-1
                if (x.isdigit() or ('-' in x)) :
                    if no-no_end <= no_final:
                        sales.append(x)
                    else:
                        break
                        
            flag_op_date = False
            
            for no, x in enumerate(ls_filtered) :
                x = x.replace(',','')
                if x == 'Operating Income':
                    no_start = no
                    flag_op_date = True
                if flag_op_date and no-no_start < no_final+2 :
                    op.append(x)


            date = date[1:no_final+1]
            sales = sales[1:]
            op = op[2:]
            df= pd.DataFrame([date, sales, op]).T
            df.columns = ['FS_DATE','SALES','OP_INCOME']
            df.insert(0, 'TICKER', tk)
            df.insert(2, 'UP_DATE', dt.datetime.today().strftime('%Y-%m-%d'))

        except:
            driver.quit()
            continue
        
        dfTTL = pd.concat([dfTTL, df])
        break

dfTTL.reset_index(drop=True, inplace=True)
dfTTL.FS_DATE = pd.to_datetime(dfTTL.FS_DATE).dt.strftime("%Y-%m-%d")

dfTTL = dfTTL.melt(id_vars=['TICKER','FS_DATE','UP_DATE'],
                   var_name='FS_ITEM',
                   value_name='VALUE'
                   )

dfTTL.to_csv('6_SP500_REV_WebCrawling.csv')
print('\ncrawling ended\n')

# mail 
if mail_flag :
    print('preparing mail\n')
    msg = MIMEMultipart()
    msg['From'] = 'iamsukwon@naver.com'
    msg['To'] = 'sw46.hong@samsung.com'
    msg['Subject'] = Header(s='SP500 REVENUE', charset='utf-8')
    body = MIMEText('첨부된 파일을 확인해 주세요.', _charset='utf-8')
    msg.attach(body)

    files = ['./6_SP500_REV_WebCrawling.csv']

    for file in files:
        part = MIMEBase('application', "octet-stream")
        part.set_payload(open(file,"rb").read())
        encode_base64(part)
        part.add_header('Content-Disposition', 'attachment; filename="%s"' % os.path.basename(file))
        msg.attach(part)

    mailServer = smtplib.SMTP('smtp.naver.com', 587)
    mailServer.starttls()
    mailServer.login('iamsukwon@naver.com', 'Okok7410!00')
    mailServer.send_message(msg)
    mailServer.quit()
    print('Done')
    bot = telegram.Bot(token='5654356034:AAE6al7LcrJEnF_D-mAhXAxV2_dfGtQ17a4')
    chat_id = 47418377
    bot.sendMessage(chat_id=chat_id, text="SP500_REV_WebCrawling Done")

S&P500 Rev Crawling:   0%|          | 0/503 [00:00<?, ?it/s]

MMM
Message: 

AOS
Message: 

ABT
Message: 

ABBV
Message: 

ACN
Message: 

ATVI
Message: 

ADM
Message: 

ADBE
Message: 

ADP
Message: 

AES
Message: 

AFL
Message: 

A
Message: 

APD
Message: 

AKAM
Message: 

ALK
Message: 

ALB
Message: 

ARE
Message: 

ALGN
Message: 

ALLE
Message: 

LNT
Message: 

ALL
Message: 

GOOGL
Message: 

GOOG
Message: 

MO
Message: 

AMZN
Message: 

AMCR
Message: 

AMD
Message: 

AEE
Message: 

AAL
Message: 

AEP
Message: 

AXP
Message: 

AIG
Message: 

AMT
Message: 

AWK
Message: 

AMP
Message: 

ABC
Message: 

AME
Message: 

AMGN
Message: 

APH
Message: 

ADI
Message: 

ANSS
Message: 

AON
Message: 

APA
Message: 

AAPL
Message: 

AMAT
Message: 

APTV
Message: 

ACGL
Message: 

ANET
Message: 

AJG
Message: 

AIZ
Message: 

T
Message: 

ATO
Message: 

ADSK
Message: 

AZO
Message: 

AVB
Message: 

AVY
Message: 

AXON
Message: 

BKR
Message: 

BALL
Message: 

BAC
Message: 

BBWI
Message: 

BAX
Message: 

BDX
Message: 

WRB
Message: 

BRK-B
Message: 

BBY
Me